In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
dataset = np.loadtxt('ex5.csv', delimiter=',')
X = dataset[:, 0:8]
y = dataset[:, 8]

Zmienne wejściowe:
Liczba ciąż
Stężenie glukozy w osoczu po 2 godzinach w doustnym teście tolerancji glukozy
Rozkurczowe ciśnienie krwi (mm Hg)
Grubość fałdu skórnego tricepsa (mm)
2-godzinne stężenie insuliny w surowicy (μIU/ml)
Wskaźnik masy ciała (waga w kg/(wzrost w m)2)
Funkcja rodowodu cukrzycy
Wiek (lata)

Y: Etykieta klasy (0 lub 1)


In [3]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [4]:

model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid())

In [5]:
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


In [6]:

# Importuj bibliotekę PyTorch.
import torch.nn as nn

# Definiuj klasę Classifier, która dziedziczy po nn.Module z PyTorch.


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()

        # Definiuj warstwy sieci neuronowej.
        # Warstwa ukryta 1 z 8 wejściami i 12 wyjściami.
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()  # Funkcja aktywacji ReLU po warstwie ukrytej 1.
        # Warstwa ukryta 2 z 12 wejściami i 8 wyjściami.
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()  # Funkcja aktywacji ReLU po warstwie ukrytej 2.
        # Warstwa wyjściowa z 8 wejściami i 1 wyjściem.
        self.output = nn.Linear(8, 1)
        # Funkcja aktywacji Sigmoid po warstwie wyjściowej.
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        # Określa przód (forward pass) sieci neuronowej.
        # Przeprowadza dane przez warstwę ukrytą 1 z ReLU.
        x = self.act1(self.hidden1(x))
        # Przeprowadza dane przez warstwę ukrytą 2 z ReLU.
        x = self.act2(self.hidden2(x))
        # Przeprowadza dane przez warstwę wyjściową z Sigmoid.
        x = self.act_output(self.output(x))
        return x


# Tworzy instancję modelu klasyfikatora.
model = Classifier()

# Wyświetla definicję modelu.
print(model)

Classifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [7]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Liczba epok i rozmiar partii (batch size) w treningu.
n_epochs = 100
batch_size = 10

# Pętla treningowa po epokach.
for epoch in range(n_epochs):
    # Pętla treningowa po danych treningowych z podziałem na partie (mini-batch).
    for i in range(0, len(X), batch_size):
        # Wybiera partię danych treningowych.
        Xbatch = X[i:i+batch_size]

        # Przewiduje wyniki za pomocą modelu.
        y_pred = model(Xbatch)

        # Wybiera odpowiadające etykiety (klasy) dla partii danych.
        ybatch = y[i:i+batch_size]

        # Oblicza stratę (loss) między przewidywaniami a rzeczywistymi etykietami.
        loss = loss_fn(y_pred, ybatch)

        # Wyzerowuje gradienty w optymalizatorze.
        optimizer.zero_grad()

        # Oblicza gradienty straty wstecz (backpropagation).
        loss.backward()

        # Aktualizuje wagi modelu na podstawie gradientów.
        optimizer.step()

    # Wyświetla informacje o postępie treningu po każdej epoce.
    print(f'Epoka {epoch}, funkcja straty {loss}')

Epoka 0, funkcja straty 0.6671252250671387
Epoka 1, funkcja straty 0.6425540447235107
Epoka 2, funkcja straty 0.6390970349311829
Epoka 3, funkcja straty 0.6318089365959167
Epoka 4, funkcja straty 0.6295920014381409
Epoka 5, funkcja straty 0.6236585974693298
Epoka 6, funkcja straty 0.6226415038108826
Epoka 7, funkcja straty 0.6156611442565918
Epoka 8, funkcja straty 0.6112000346183777
Epoka 9, funkcja straty 0.602994978427887
Epoka 10, funkcja straty 0.6101362109184265
Epoka 11, funkcja straty 0.5923745036125183
Epoka 12, funkcja straty 0.6061577200889587
Epoka 13, funkcja straty 0.5833727121353149
Epoka 14, funkcja straty 0.5875617265701294
Epoka 15, funkcja straty 0.5837463140487671
Epoka 16, funkcja straty 0.5793173313140869
Epoka 17, funkcja straty 0.5722836256027222
Epoka 18, funkcja straty 0.5535190105438232
Epoka 19, funkcja straty 0.553154706954956
Epoka 20, funkcja straty 0.5466154217720032
Epoka 21, funkcja straty 0.5617343187332153
Epoka 22, funkcja straty 0.5578280687332153


In [9]:
# Ustaw tryb ewaluacji (bez obliczania gradientów) za pomocą torch.no_grad().
with torch.no_grad():
    # Przewiduje wyniki za pomocą modelu na całym zbiorze danych treningowych.
    y_pred = model(X)

# Oblicza dokładność (accuracy) modelu na danych treningowych.
# Dokładność to procent poprawnie sklasyfikowanych przykładów.
accuracy = (y_pred.round() == y).float().mean()

# Wyświetla wynik dokładności.
print(f"Accuracy {accuracy}")

Accuracy 0.7473958134651184


In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


dataset = np.loadtxt('ex5.csv', delimiter=',')
X = dataset[:, 0:8]
y = dataset[:, 8]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)


model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid()
)
print(model)


loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoka {epoch}, funkcja straty {loss}')

with torch.no_grad():
    y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)
Epoka 0, funkcja straty 0.6356704831123352
Epoka 1, funkcja straty 0.591513991355896
Epoka 2, funkcja straty 0.5675687193870544
Epoka 3, funkcja straty 0.553988516330719
Epoka 4, funkcja straty 0.5455507040023804
Epoka 5, funkcja straty 0.5415869951248169
Epoka 6, funkcja straty 0.5392428636550903
Epoka 7, funkcja straty 0.5365587472915649
Epoka 8, funkcja straty 0.5373504757881165
Epoka 9, funkcja straty 0.5395587086677551
Epoka 10, funkcja straty 0.5424097180366516
Epoka 11, funkcja straty 0.5407059788703918
Epoka 12, funkcja straty 0.5418652296066284
Epoka 13, funkcja straty 0.5394490361213684
Epoka 14, funkcja straty 0.541008710861206
Epoka 15, funkcja straty 0.5395030379295349
Epoka 16, funkcja straty 0.5443186163902283
Epoka 17, funkcja straty 0.540149807

In [11]:

predictions = model(X)

rounded = predictions.round()

In [12]:

predictions = (model(X) > 0.5).int()

In [13]:
predictions = (model(X) > 0.5).int()
for i in range(10):
    print('%s => %d (oczekiwane %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.599998474121094, 0.6269999742507935, 50.0] => 1 (oczekiwane 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.600000381469727, 0.35100001096725464, 31.0] => 0 (oczekiwane 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.299999237060547, 0.671999990940094, 32.0] => 1 (oczekiwane 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.100000381469727, 0.16699999570846558, 21.0] => 0 (oczekiwane 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.099998474121094, 2.2880001068115234, 33.0] => 1 (oczekiwane 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.600000381469727, 0.20100000500679016, 30.0] => 0 (oczekiwane 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.24799999594688416, 26.0] => 0 (oczekiwane 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.29999923706055, 0.1340000033378601, 29.0] => 1 (oczekiwane 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.15800000727176666, 53.0] => 1 (oczekiwane 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.23199999332427979, 54.0] => 0 (oczekiwane 1)
